# 1. Import & Data load

In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import eli5
from eli5.sklearn import PermutationImportance
from scipy.stats import skew
from scipy.stats import kurtosis
from sklearn.cluster import KMeans
import math
pi = math.pi
pd.options.display.max_columns=500

train=pd.read_csv('./raw/train_features.csv')
y=pd.read_csv('./raw//train_labels.csv')
test=pd.read_csv('./raw//test_features.csv')
sub=pd.read_csv('./raw//sample_submission.csv')

# 2. Custom function

In [2]:
def range_func(x):
    max_val = np.max(x)
    min_val = np.min(x)
    range_val = max_val - min_val
    return range_val

def iqr_func2(x):
    q3, q1 = np.percentile(x, [20, 80])
    iqr = q3 - q1
    return iqr

def iqr_func3(x):
    q3, q1 = np.percentile(x, [40, 60])
    iqr = q3 - q1
    return iqr

def iqr_func4(x):
    q3, q1 = np.percentile(x, [15, 95])
    iqr = q3 - q1
    return iqr

def premad(x):
    return np.median(np.absolute(x-np.median(x,axis=0)),axis=0)

def preskew(x):
    return skew(x)

def prekurt(x):
    return kurtosis(x,fisher=True)

# 3. Create change data
- 시간 변동에 따른 수치 변화 데이터

In [4]:
train_change=pd.DataFrame()
change_train=train.set_index('id').drop('time',axis=1)
for j in train['id'].unique():
    for i in range(599):
        train_change=train_change.append(change_train[change_train.index==j].iloc[i+1,:]-change_train[change_train.index==j].iloc[i,:])

test_change=pd.DataFrame()
change_test=test.set_index('id').drop('time',axis=1)
for j in test['id'].unique():
    for i in range(599):
        test_change=test_change.append(change_test[change_test.index==j].iloc[i+1,:]-change_test[change_test.index==j].iloc[i,:])

# 4. Feature engineering

### Add vectors

In [ ]:
train['acc_vector']=np.sqrt((train['acc_x'] ** 2) +(train['acc_y'] ** 2)+(train['acc_z'] ** 2))
train['gy_vector']=np.sqrt((train['gy_x'] ** 2) +(train['gy_y'] ** 2)+(train['gy_z'] ** 2))

test['acc_vector']=np.sqrt((test['acc_x'] ** 2) +(test['acc_y'] ** 2)+(test['acc_z'] ** 2))
test['gy_vector']=np.sqrt((test['gy_x'] ** 2) +(test['gy_y'] ** 2)+(test['gy_z'] ** 2))

train['acc_YZvector']=np.sqrt((train['acc_y'] ** 2) +(train['acc_z'] ** 2))
train['gy_YZvector']=np.sqrt((train['gy_y'] ** 2) +(train['gy_z'] ** 2))

train['acc_XYvector']=np.sqrt((train['acc_x'] ** 2) +(train['acc_y'] ** 2))
train['gy_XYvector']=np.sqrt((train['gy_x'] ** 2) +(train['gy_y'] ** 2))

train['acc_XZvector']=np.sqrt((train['acc_x'] ** 2) +(train['acc_z'] ** 2))
train['gy_XZvector']=np.sqrt((train['gy_x'] ** 2) +(train['gy_z'] ** 2))

test['acc_YZvector']=np.sqrt((test['acc_y'] ** 2) +(test['acc_z'] ** 2))
test['gy_YZvector']=np.sqrt((test['gy_y'] ** 2) +(test['gy_z'] ** 2))

test['acc_XYvector']=np.sqrt((test['acc_x'] ** 2) +(test['acc_y'] ** 2))
test['gy_XYvector']=np.sqrt((test['gy_x'] ** 2) +(test['gy_y'] ** 2))

test['acc_XZvector']=np.sqrt((test['acc_x'] ** 2) +(test['acc_z'] ** 2))
test['gy_XZvector']=np.sqrt((test['gy_x'] ** 2) +(test['gy_z'] ** 2))

### Add center of gravity

In [ ]:
# 자이로스코프 무게중심
train['gy_Centerofgravity']=(train['gy_x']+train['gy_y']+train['gy_z'])/3
test['gy_Centerofgravity']=(test['gy_x']+test['gy_y']+test['gy_z'])/3

### Add roll & pitch

In [3]:
# roll & pitch
train['roll'] = np.arctan(train['acc_y']/np.sqrt(train['acc_x'] ** 2 + train['acc_z'] ** 2))
test['roll'] = np.arctan(test['acc_y']/np.sqrt(test['acc_x'] ** 2 + test['acc_z'] ** 2))

train['pitch'] = np.arctan(train['acc_x']/np.sqrt(train['acc_y'] ** 2 + train['acc_z'] ** 2))
test['pitch'] = np.arctan(test['acc_x']/np.sqrt(test['acc_y'] ** 2 + test['acc_z'] ** 2))

train['math_roll'] = np.arctan(- train['acc_x']/np.sqrt(train['acc_y'] ** 2 + train['acc_z'] ** 2)) * (180/pi)
train['math_pitch'] = np.arctan(train['acc_y']/np.sqrt(train['acc_x'] ** 2 + train['acc_z'] ** 2)) * (180/pi)

test['math_roll'] = np.arctan(- test['acc_x']/np.sqrt(test['acc_y'] ** 2 + test['acc_z'] ** 2)) * (180/pi)
test['math_pitch'] = np.arctan(test['acc_y']/np.sqrt(test['acc_x'] ** 2 + test['acc_z'] ** 2)) * (180/pi)

train['gy_roll'] = np.arctan(train['gy_y']/np.sqrt(train['gy_x'] ** 2 + train['gy_z'] ** 2))
test['gy_roll'] = np.arctan(test['gy_y']/np.sqrt(test['gy_x'] ** 2 + test['gy_z'] ** 2))

train['gy_pitch'] = np.arctan(train['gy_x']/np.sqrt(train['gy_y'] ** 2 + train['gy_z'] ** 2))
test['gy_pitch'] = np.arctan(test['gy_x']/np.sqrt(test['gy_y'] ** 2 + test['gy_z'] ** 2))

train['gy_math_roll'] = np.arctan(- train['gy_x']/np.sqrt(train['gy_y'] ** 2 + train['gy_z'] ** 2)) * (180/pi)
train['gy_math_pitch'] = np.arctan(train['gy_y']/np.sqrt(train['gy_x'] ** 2 + train['gy_z'] ** 2)) * (180/pi)

test['gy_math_roll'] = np.arctan(- test['gy_x']/np.sqrt(test['gy_y'] ** 2 + test['gy_z'] ** 2)) * (180/pi)
test['gy_math_pitch'] = np.arctan(test['gy_y']/np.sqrt(test['gy_x'] ** 2 + test['gy_z'] ** 2)) * (180/pi)

### Apply the custom function

In [62]:
features = ['id', 'acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z','acc_vector','gy_vector'
            ,'acc_YZvector','gy_YZvector','acc_XYvector','gy_XYvector','acc_XZvector','gy_XZvector','gy_Centerofgravity']
features2 = ['id', 'roll','pitch','math_roll','math_pitch', 'gy_roll','gy_pitch','gy_math_roll','gy_math_pitch']

train_preprocess = train[features].groupby('id').agg(['max', 'min', 'mean','std','median',range_func,iqr_func2,iqr_func3,iqr_func4,premad,preskew,prekurt])
temp_train_preprocess = train[features2].groupby('id').agg([range_func,iqr_func2,iqr_func3,iqr_func4,premad,preskew,prekurt])
test_preprocess = test[features].groupby('id').agg(['max', 'min', 'mean','std','median',range_func,iqr_func2,iqr_func3,iqr_func4,premad,preskew,prekurt])
temp_test_preprocess = test[features2].groupby('id').agg([range_func,iqr_func2,iqr_func3,iqr_func4,premad,preskew,prekurt])

train_preprocess=pd.concat([train_preprocess,temp_train_preprocess],axis=1)
test_preprocess=pd.concat([test_preprocess,temp_test_preprocess],axis=1)

train_preprocess.columns=[i[0]+'_'+i[1] for i in train_preprocess.columns]
test_preprocess.columns=[i[0]+'_'+i[1] for i in test_preprocess.columns]

### Add std mean

In [63]:
train_preprocess['acc_std_mean']=(train_preprocess['acc_x_std']+train_preprocess['acc_y_std']+ train_preprocess['acc_z_std'])/3
train_preprocess['gy_std_mean']=(train_preprocess['gy_x_std']+train_preprocess['gy_y_std']+ train_preprocess['gy_z_std'])/3

test_preprocess['acc_std_mean']=(test_preprocess['acc_x_std']+test_preprocess['acc_y_std']+ test_preprocess['acc_z_std'])/3
test_preprocess['gy_std_mean']=(test_preprocess['gy_x_std']+test_preprocess['gy_y_std']+ test_preprocess['gy_z_std'])/3

### Add the front part of data

In [65]:
# 가속도계 첫번째 데이터
train_acc_head1=pd.DataFrame()
for i in train['id'].unique():
    train_acc_head1=train_acc_head1.append(train[['id','acc_x','acc_y','acc_z']][train[['id','acc_x','acc_y','acc_z']]['id']==i].head(1))
train_acc_head1.columns=['id','first_acc_x','first_acc_y','first_acc_z']
train_acc_head1.set_index('id',inplace=True)

test_acc_head1=pd.DataFrame()
for i in test['id'].unique():
    test_acc_head1=test_acc_head1.append(test[['id','acc_x','acc_y','acc_z']][test[['id','acc_x','acc_y','acc_z']]['id']==i].head(1))
test_acc_head1.columns=['id','first_acc_x','first_acc_y','first_acc_z']
test_acc_head1.set_index('id',inplace=True)

# 가속도계 첫 3초
train_acc_head=pd.DataFrame()
for i in train['id'].unique():
    train_acc_head=train_acc_head.append(train[['id','acc_x','acc_y','acc_z']][train[['id','acc_x','acc_y','acc_z']]['id']==i].head(150))
train_acc_head.columns=['id','head_acc_x','head_acc_y','head_acc_z']
train_acc_head=train_acc_head.groupby('id').mean()

test_acc_head=pd.DataFrame()
for i in test['id'].unique():
    test_acc_head=test_acc_head.append(test[['id','acc_x','acc_y','acc_z']][test[['id','acc_x','acc_y','acc_z']]['id']==i].head(150))
test_acc_head.columns=['id','head_acc_x','head_acc_y','head_acc_z']
test_acc_head=test_acc_head.groupby('id').mean()

train_preprocess=pd.concat([train_preprocess,train_acc_head,train_acc_head1],axis=1)
test_preprocess=pd.concat([test_preprocess,test_acc_head,test_acc_head1],axis=1)

# 자이로스코프 첫 3초
train_gy_head=pd.DataFrame()
for i in train['id'].unique():
    train_gy_head=train_gy_head.append(train[['id','gy_x','gy_y','gy_z']][train[['id','gy_x','gy_y','gy_z']]['id']==i].head(150))
train_gy_head.columns=['id','head_gy_x','head_gy_y','head_gy_z']
train_gy_head=train_gy_head.groupby('id').mean()

test_gy_head=pd.DataFrame()
for i in test['id'].unique():
    test_gy_head=test_gy_head.append(test[['id','gy_x','gy_y','gy_z']][test[['id','gy_x','gy_y','gy_z']]['id']==i].head(150))
test_gy_head.columns=['id','head_gy_x','head_gy_y','head_gy_z']
test_gy_head=test_gy_head.groupby('id').mean()

train_preprocess=pd.concat([train_preprocess,train_gy_head],axis=1)
test_preprocess=pd.concat([test_preprocess,test_gy_head],axis=1)

### Add change data

In [77]:
# train 변화량 데이터 추가
temp=train_change[['id','acc_x','acc_y','acc_z']]
temp.columns=['id','Cacc_x','Cacc_y','Cacc_z']
temp['Croll'] = np.arctan(temp['Cacc_y']/np.sqrt(temp['Cacc_x'] ** 2 + temp['Cacc_z'] ** 2))
temp['Cpitch'] = np.arctan(temp['Cacc_x']/np.sqrt(temp['Cacc_y'] ** 2 + temp['Cacc_z'] ** 2))
temp['Cmath_roll'] = np.arctan(- temp['Cacc_x']/np.sqrt(temp['Cacc_y'] ** 2 + temp['Cacc_z'] ** 2)) * (180/pi)
temp['Cmath_pitch'] = np.arctan(temp['Cacc_y']/np.sqrt(temp['Cacc_x'] ** 2 + temp['Cacc_z'] ** 2)) * (180/pi)
temp=temp.groupby('id').agg([range_func,iqr_func2,iqr_func3,iqr_func4,premad,preskew,prekurt])
train_preprocess=pd.concat([train_preprocess,temp],axis=1)

temp=train_change[['id','gy_x','gy_y','gy_z']]
temp.columns=['id','Cgy_x','Cgy_y','Cgy_z']
temp['Cgyroll'] = np.arctan(temp['Cgy_y']/np.sqrt(temp['Cgy_x'] ** 2 + temp['Cgy_z'] ** 2))
temp['Cgypitch'] = np.arctan(temp['Cgy_x']/np.sqrt(temp['Cgy_y'] ** 2 + temp['Cgy_z'] ** 2))
temp['Cgymath_roll'] = np.arctan(- temp['Cgy_x']/np.sqrt(temp['Cgy_y'] ** 2 + temp['Cgy_z'] ** 2)) * (180/pi)
temp['Cgymath_pitch'] = np.arctan(temp['Cgy_y']/np.sqrt(temp['Cgy_x'] ** 2 + temp['Cgy_z'] ** 2)) * (180/pi)
temp=temp.groupby('id').agg([range_func,iqr_func2,iqr_func3,iqr_func4,premad,preskew,prekurt])
train_preprocess=pd.concat([train_preprocess,temp],axis=1)

# test 변화량 데이터 추가
temp=test_change[['id','acc_x','acc_y','acc_z']]
temp.columns=['id','Cacc_x','Cacc_y','Cacc_z']
temp['Croll'] = np.arctan(temp['Cacc_y']/np.sqrt(temp['Cacc_x'] ** 2 + temp['Cacc_z'] ** 2))
temp['Cpitch'] = np.arctan(temp['Cacc_x']/np.sqrt(temp['Cacc_y'] ** 2 + temp['Cacc_z'] ** 2))
temp['Cmath_roll'] = np.arctan(- temp['Cacc_x']/np.sqrt(temp['Cacc_y'] ** 2 + temp['Cacc_z'] ** 2)) * (180/pi)
temp['Cmath_pitch'] = np.arctan(temp['Cacc_y']/np.sqrt(temp['Cacc_x'] ** 2 + temp['Cacc_z'] ** 2)) * (180/pi)
temp=temp.groupby('id').agg([range_func,iqr_func2,iqr_func3,iqr_func4,premad,preskew,prekurt])
test_preprocess=pd.concat([test_preprocess,temp],axis=1)

temp=test_change[['id','gy_x','gy_y','gy_z']]
temp.columns=['id','Cgy_x','Cgy_y','Cgy_z']
temp['Cgyroll'] = np.arctan(temp['Cgy_y']/np.sqrt(temp['Cgy_x'] ** 2 + temp['Cgy_z'] ** 2))
temp['Cgypitch'] = np.arctan(temp['Cgy_x']/np.sqrt(temp['Cgy_y'] ** 2 + temp['Cgy_z'] ** 2))
temp['Cgymath_roll'] = np.arctan(- temp['Cgy_x']/np.sqrt(temp['Cgy_y'] ** 2 + temp['Cgy_z'] ** 2)) * (180/pi)
temp['Cgymath_pitch'] = np.arctan(temp['Cgy_y']/np.sqrt(temp['Cgy_x'] ** 2 + temp['Cgy_z'] ** 2)) * (180/pi)
temp=temp.groupby('id').agg([range_func,iqr_func2,iqr_func3,iqr_func4,premad,preskew,prekurt])
test_preprocess=pd.concat([test_preprocess,temp],axis=1)

### Clustering - KMeans

In [67]:
model = KMeans(n_clusters=5,random_state=20)
model.fit(train_preprocess)
train_predict = model.predict(train_preprocess)
train_preprocess['cluster']=train_predict

test_predict = model.predict(test_preprocess)
test_preprocess['cluster']=test_predict

### Update columns name

In [68]:
column_name=list(train_preprocess.iloc[:,:247].columns)
column_name.extend([i[0]+ '-' + i[1] for i in train_preprocess.iloc[:,247:-1].columns])
column_name.extend(list(train_preprocess.iloc[:,-1:].columns))

train_preprocess.columns=column_name
test_preprocess.columns=column_name

In [76]:
train_preprocess.head(2)

,acc_x_max,acc_x_min,acc_x_mean,acc_x_std,acc_x_median,acc_x_range_func,acc_x_iqr_func2,acc_x_iqr_func3,acc_x_iqr_func4,acc_x_premad,acc_x_preskew,acc_x_prekurt,acc_y_max,acc_y_min,acc_y_mean,acc_y_std,acc_y_median,acc_y_range_func,acc_y_iqr_func2,acc_y_iqr_func3,acc_y_iqr_func4,acc_y_premad,acc_y_preskew,acc_y_prekurt,acc_z_max,acc_z_min,acc_z_mean,acc_z_std,acc_z_median,acc_z_range_func,acc_z_iqr_func2,acc_z_iqr_func3,acc_z_iqr_func4,acc_z_premad,acc_z_preskew,acc_z_prekurt,gy_x_max,gy_x_min,gy_x_mean,gy_x_std,gy_x_median,gy_x_range_func,gy_x_iqr_func2,gy_x_iqr_func3,gy_x_iqr_func4,gy_x_premad,gy_x_preskew,gy_x_prekurt,gy_y_max,gy_y_min,gy_y_mean,gy_y_std,gy_y_median,gy_y_range_func,gy_y_iqr_func2,gy_y_iqr_func3,gy_y_iqr_func4,gy_y_premad,gy_y_preskew,gy_y_prekurt,gy_z_max,gy_z_min,gy_z_mean,gy_z_std,gy_z_median,gy_z_range_func,gy_z_iqr_func2,gy_z_iqr_func3,gy_z_iqr_func4,gy_z_premad,gy_z_preskew,gy_z_prekurt,acc_vector_max,acc_vector_min,acc_vector_mean,acc_vector_std,acc_vector_median,acc_vector_range_func,acc_vector_iqr_func2,acc_vector_iqr_func3,acc_vector_iqr_func4,acc_vector_premad,acc_vector_preskew,acc_vector_prekurt,gy_vector_max,gy_vector_min,gy_vector_mean,gy_vector_std,gy_vector_median,gy_vector_range_func,gy_vector_iqr_func2,gy_vector_iqr_func3,gy_vector_iqr_func4,gy_vector_premad,gy_vector_preskew,gy_vector_prekurt,acc_YZvector_max,acc_YZvector_min,acc_YZvector_mean,acc_YZvector_std,acc_YZvector_median,acc_YZvector_range_func,acc_YZvector_iqr_func2,acc_YZvector_iqr_func3,acc_YZvector_iqr_func4,acc_YZvector_premad,acc_YZvector_preskew,acc_YZvector_prekurt,gy_YZvector_max,gy_YZvector_min,gy_YZvector_mean,gy_YZvector_std,gy_YZvector_median,gy_YZvector_range_func,gy_YZvector_iqr_func2,gy_YZvector_iqr_func3,gy_YZvector_iqr_func4,gy_YZvector_premad,gy_YZvector_preskew,gy_YZvector_prekurt,acc_XYvector_max,acc_XYvector_min,acc_XYvector_mean,acc_XYvector_std,acc_XYvector_median,acc_XYvector_range_func,acc_XYvector_iqr_func2,acc_XYvector_iqr_func3,acc_XYvector_iqr_func4,acc_XYvector_premad,acc_XYvector_preskew,acc_XYvector_prekurt,gy_XYvector_max,gy_XYvector_min,gy_XYvector_mean,gy_XYvector_std,gy_XYvector_median,gy_XYvector_range_func,gy_XYvector_iqr_func2,gy_XYvector_iqr_func3,gy_XYvector_iqr_func4,gy_XYvector_premad,gy_XYvector_preskew,gy_XYvector_prekurt,acc_XZvector_max,acc_XZvector_min,acc_XZvector_mean,acc_XZvector_std,acc_XZvector_median,acc_XZvector_range_func,acc_XZvector_iqr_func2,acc_XZvector_iqr_func3,acc_XZvector_iqr_func4,acc_XZvector_premad,acc_XZvector_preskew,acc_XZvector_prekurt,gy_XZvector_max,gy_XZvector_min,gy_XZvector_mean,gy_XZvector_std,gy_XZvector_median,gy_XZvector_range_func,gy_XZvector_iqr_func2,gy_XZvector_iqr_func3,gy_XZvector_iqr_func4,gy_XZvector_premad,gy_XZvector_preskew,gy_XZvector_prekurt,gy_Centerofgravity_max,gy_Centerofgravity_min,gy_Centerofgravity_mean,gy_Centerofgravity_std,gy_Centerofgravity_median,gy_Centerofgravity_range_func,gy_Centerofgravity_iqr_func2,gy_Centerofgravity_iqr_func3,gy_Centerofgravity_iqr_func4,gy_Centerofgravity_premad,gy_Centerofgravity_preskew,gy_Centerofgravity_prekurt,roll_range_func,roll_iqr_func2,roll_iqr_func3,roll_iqr_func4,roll_premad,roll_preskew,roll_prekurt,pitch_range_func,pitch_iqr_func2,pitch_iqr_func3,pitch_iqr_func4,pitch_premad,pitch_preskew,pitch_prekurt,math_roll_range_func,math_roll_iqr_func2,math_roll_iqr_func3,math_roll_iqr_func4,math_roll_premad,math_roll_preskew,math_roll_prekurt,math_pitch_range_func,math_pitch_iqr_func2,math_pitch_iqr_func3,math_pitch_iqr_func4,math_pitch_premad,math_pitch_preskew,math_pitch_prekurt,gy_roll_range_func,gy_roll_iqr_func2,gy_roll_iqr_func3,gy_roll_iqr_func4,gy_roll_premad,gy_roll_preskew,gy_roll_prekurt,gy_pitch_range_func,gy_pitch_iqr_func2,gy_pitch_iqr_func3,gy_pitch_iqr_func4,gy_pitch_premad,gy_pitch_preskew,gy_pitch_prekurt,gy_math_roll_range_func,gy_math_roll_iqr_func2,gy_math_roll_iqr_func3,gy_math_roll_iqr_func4,gy_math_roll_premad,gy_math_roll_preskew,gy_math_roll_prekurt,gy_math_pitch_range_f

# 5. Fit & Predict

### Drop bad features
- permutation importance를 이용하였습니다.

In [ ]:
drop_columns=['gy_roll_range_func','gy_XZvector_iqr_func3','gy_XZvector_median','gy_Centerofgravity_prekurt']

### Create 3 catboost model
- depth의 차이를 두었습니다.

In [ ]:
cat_depth3=CatBoostClassifier(random_state=20,iterations=26000,task_type='GPU',depth=3,bootstrap_type ='Bernoulli',learning_rate = 0.007932)
cat_depth3.fit(train_preprocess.drop(drop_columns,axis=1), y['label'])
predict_depth3=cat_depth3.predict_proba(test_preprocess.drop(drop_columns,axis=1))

cat_depth4=CatBoostClassifier(random_state=20,iterations=22000,task_type='GPU',depth=4,bootstrap_type ='Bernoulli',learning_rate = 0.007932)
cat_depth4.fit(train_preprocess.drop(drop_columns,axis=1), y['label'])
predict_depth4=cat_depth4.predict_proba(test_preprocess.drop(drop_columns,axis=1))

cat_depth5=CatBoostClassifier(random_state=20,iterations=13000,task_type='GPU',depth=5,bootstrap_type ='Bernoulli')
cat_depth5.fit(train_preprocess.drop(drop_columns,axis=1), y['label'])
predict_depth5=cat_depth5.predict_proba(test_preprocess.drop(drop_columns,axis=1))

### Soft voting

In [ ]:
predict= (predict_depth3 * 0.45) + (predict_depth4 * 0.35) + (predict_depth5 * 0.2)

### Submission

In [ ]:
sub.iloc[:,1:]=predict
sub.to_csv('[final]Not_StratifiedKfold.csv', index=False)
sub